# csdid with RC/Panel

First. Some other tools. Please copy
[this](https://friosavila.github.io/stpackages/fra/fra.ado) file, run
it, and type:

``` stata
** just for data example
ssc install frause
** for installing csdid2
fra install fra
fra install csdid2
```

Now the example.

First Run csdid with panel data

In [1]:
frause mpdta, clear
set seed 1
csdid lemp, ivar( countyreal) gvar(first) time(year)

(Written by R.              )
............
Difference-in-difference with Multiple Time Periods

                                                         Number of obs = 2,500
Outcome model  : regression adjustment
Treatment model: none
------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
g2004        |
 t_2003_2004 |  -.0105032    .023251    -0.45   0.651    -.0560744    .0350679
 t_2003_2005 |  -.0704232   .0309848    -2.27   0.023    -.1311522   -.0096941
 t_2003_2006 |  -.1372587   .0364357    -3.77   0.000    -.2086713   -.0658461
 t_2003_2007 |  -.1008114   .0343592    -2.93   0.003    -.1681542   -.0334685
-------------+----------------------------------------------------------------
g2006        |
 t_2003_2004 |   .0065201   .0233268     0.28   0.780    -.0391996    .0522398
 t_2004_2005 |  -.00275

Then, drop at randome 10% of the data, so technically have unbalanced
panel. It can be estimated using repeated crossection

In [2]:
drop if runiform()<.1
** as Panel with unbalanced Data
csdid lemp,  ivar( countyreal) gvar(first) time(year)

(239 observations deleted)
Panel is not balanced
Will use observations with Pair balanced (observed at t0 and t1)
............
Difference-in-difference with Multiple Time Periods

                                                         Number of obs = 2,227
Outcome model  : regression adjustment
Treatment model: none
------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
g2004        |
 t_2003_2004 |  -.0062926   .0225497    -0.28   0.780    -.0504892     .037904
 t_2003_2005 |  -.0960718   .0357584    -2.69   0.007    -.1661569   -.0259866
 t_2003_2006 |  -.1633641   .0417271    -3.92   0.000    -.2451478   -.0815804
 t_2003_2007 |  -.1295976     .03268    -3.97   0.000    -.1936492   -.0655459
-------------+----------------------------------------------------------------
g2006        |
 t_2003_2004 |  -.

In [3]:
** As RC
csdid lemp,  cluster( countyreal) gvar(first) time(year)

............
Difference-in-difference with Multiple Time Periods

                                                         Number of obs = 2,261
Outcome model  : regression adjustment
Treatment model: none
                           (Std. err. adjusted for 500 clusters in countyreal)
------------------------------------------------------------------------------
             | Coefficient  Std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
g2004        |
 t_2003_2004 |  -.1173444   .1395652    -0.84   0.400    -.3908872    .1561983
 t_2003_2005 |  -.2553488   .2439786    -1.05   0.295     -.733538    .2228404
 t_2003_2006 |  -.3608912   .1722595    -2.10   0.036    -.6985137   -.0232687
 t_2003_2007 |   -.305137    .112898    -2.70   0.007     -.526413    -.083861
-------------+----------------------------------------------------------------
g2006        |
 t_2003_2004 |   .1768316   .0896097     1.97   0.048

The second one produces odd results, which is not unexpected.

Now using the residuals Idea:

In [4]:
** Getting residuals
reghdfe lemp, abs( year countyreal) resid
** AS RC but with residuals
csdid _reghdfe_resid, cluster( countyreal) gvar(first) time(year)

(dropped 1 singleton observations)
(MWFE estimator converged in 5 iterations)

HDFE Linear regression                            Number of obs   =      2,260
Absorbing 2 HDFE groups                           F(   0,   1757) =          .
                                                  Prob > F        =          .
                                                  R-squared       =     0.9935
                                                  Adj R-squared   =     0.9916
                                                  Within R-sq.    =     0.0000
                                                  Root MSE        =     0.1378

------------------------------------------------------------------------------
        lemp | Coefficient  Std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
       _cons |    5.78452   .0028984  1995.80   0.000     5.778835    5.790205
---------------------------------------------------

I think this produces much better results! Not sure how would this
translate in other frameworks, tho.